In [0]:
import mxnet as mx
import random
import numpy as np

Class for implementation of linear regression

In [0]:
class LinearRegression():

  def __init__(self, weights, bias ):

    """
    """

    self.weights = weights
    self.bias = bias


  def synthetic_data(self, w, b, size):
    """
    weights: synethic_weight nd array

    bias: fixed value 

    size: the size of data set we want to create

    returns genearted data

    """

    # Genearte X with normal distribution

    X = mx.nd.normal(0,  0.1, (size, len(w)),
                     ctx = mx.gpu())
    
    
    y = mx.nd.dot(X, w) + b # add bias

    #adding additional noise which is normally distribued centered at 0 and variance : 0.1

    y = y + mx.nd.normal(0, 0.1, y.shape,
                         ctx = mx.gpu()) 

    return X,y

  def data_iter(self , batch_size, features, labels):

    """
    batch_size:  min batch size

    features: X matrix

    labels: y

    returns: X,y pairs  in mini batches 
    """

    num_examples = len(features)
    indices = list(range(num_examples))
    random.shuffle(indices) # shuffle indices to pick randomly for each batch
    batch_indcies = mx.nd.array([], mx.gpu())


    #  create batches of batch_size  
    for i in range(0, num_examples, batch_size):

      batch_indices = mx.nd.array( indices[i: min(i+batch_size, num_examples)], mx.gpu() )

      yield features[batch_indices], labels[batch_indices]

  def linreg(self, parameters, X):

    """
    Linear regression equation


    X : m x n matrix

    w : n x n matrix

    b : bias term
    """

    y = mx.nd.dot(X, parameters[0]) + parameters[1] 

    return y

  def squared_loss( self, y_hat, y):

    """

    y_hat : estimate of y

    y : actual value of y

    returns: squared loss 
    """

    return (y_hat - y.reshape(y_hat.shape))**2 / 2

  def sgd(self,parameters, lr, batch_size):

    """
    Stochastic gradient descent

    parameters: model parameters
    lr : learning rate
    batch_Size : mini batch size
    returns: updated parameters  
    """
    for param in parameters:
      param[:] = param - lr * param.grad / batch_size
    # self.weights = self.weights - lr*self.weights / batch_size
    # self.bias = self.bias - lr*self.bias /batch_size 
    


  def train(self, epochs, features, labels, batch_size, lr):
    """
    """

    self.weights.attach_grad()
    self.bias.attach_grad()

    for epoch in range(epochs):


      for X,y in self.data_iter(batch_size, features, labels):


        with mx.autograd.record():
          loss_batch = self.squared_loss(self.linreg([self.weights, self.bias], X), y)

        loss_batch.backward()

        

        self.sgd([self.weights, self.bias], lr, batch_size)

      train_l = self.squared_loss(lrobj.linreg([self.weights, self.bias], features), labels)

      print('epoch %d, loss %f' % (epoch + 1, train_l.mean().asnumpy()))

      print(self.weights)


    


 












Training of the model

In [16]:


### Initialize model parameters
w = mx.nd.random.normal(0, 0.01, (2, 1), ctx = mx.gpu()) 
b = mx.nd.zeros(1, ctx = mx.gpu())

lrobj = LinearRegression(w,b)
### genearte data
 
true_w = mx.nd.array([2, -3.4], mx.gpu())
true_b = 4.2
data = lrobj.synthetic_data( true_w, true_b, 1000)

### Allocate space to gradients of w and b


### defining epochs, learning rates and batch sizes

epochs = 100
lr = 0.1
batch_size = 10
features = data[0]
labels = data[1]

lrobj.train( epochs, features, labels, batch_size, lr)


# for epoch in range(epochs):

#   print(w,b)

#   for X,y in lrobj.data_iter(batch_size, features, labels):


#     with mx.autograd.record():
#       loss_batch = lrobj.squared_loss(lrobj.linreg(X,w,b), y)

#     loss_batch.backward()

    

#     lrobj.sgd([w,b], lr, batch_size)

#   train_l = lrobj.squared_loss(lrobj.linreg(features, w, b), labels)

#   print('epoch %d, loss %f' % (epoch + 1, train_l.mean().asnumpy()))






epoch 1, loss 0.064864

[[ 0.20256732]
 [-0.34180346]]
<NDArray 2x1 @gpu(0)>
epoch 2, loss 0.055267

[[ 0.36228392]
 [-0.61755323]]
<NDArray 2x1 @gpu(0)>
epoch 3, loss 0.046140

[[ 0.50766987]
 [-0.869848  ]]
<NDArray 2x1 @gpu(0)>
epoch 4, loss 0.039041

[[ 0.64071906]
 [-1.0993583 ]]
<NDArray 2x1 @gpu(0)>
epoch 5, loss 0.033326

[[ 0.76151705]
 [-1.3079939 ]]
<NDArray 2x1 @gpu(0)>
epoch 6, loss 0.028491

[[ 0.8719064]
 [-1.4974056]]
<NDArray 2x1 @gpu(0)>
epoch 7, loss 0.024482

[[ 0.9729746]
 [-1.6694366]]
<NDArray 2x1 @gpu(0)>
epoch 8, loss 0.021214

[[ 1.0642432]
 [-1.8261853]]
<NDArray 2x1 @gpu(0)>
epoch 9, loss 0.018600

[[ 1.1486788]
 [-1.9683475]]
<NDArray 2x1 @gpu(0)>
epoch 10, loss 0.016192

[[ 1.2251072]
 [-2.0972092]]
<NDArray 2x1 @gpu(0)>
epoch 11, loss 0.014337

[[ 1.2950623]
 [-2.2145708]]
<NDArray 2x1 @gpu(0)>
epoch 12, loss 0.012820

[[ 1.3590178]
 [-2.3207161]]
<NDArray 2x1 @gpu(0)>
epoch 13, loss 0.011595

[[ 1.4174912]
 [-2.4171684]]
<NDArray 2x1 @gpu(0)>
epoch 14, l

error in estimated weights vs actual weights

In [17]:
print('Error in estimating w', true_w - lrobj.weights.reshape(true_w.shape))
print('Error in estimating b', true_b - lrobj.bias)

Error in estimating w 
[-0.02602124 -0.02029443]
<NDArray 2 @gpu(0)>
Error in estimating b 
[0.00297785]
<NDArray 1 @gpu(0)>
